In [ ]:
# default_exp searchCounter

# SearchCounter

> DynamoDB table/class definitions for recording searches

In [ ]:
#export
from dataclasses import dataclass, field
from typing import List
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, JSONAttribute, NumberAttribute, ListAttribute
from nicHelper.datetime import datestamp as ds
from datetime import datetime
import pytz, time

In [ ]:
#export
from src.getEnv import DAXENDPOINT, BRANCH

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the GetSecretValue operation: Secrets Manager can't find the specified secret.

In [ ]:
#hide
BRANCH = "fakeBranch"
DAXENDPOINT = None

In [ ]:
#export
@dataclass
class SearchResultRecord:
    cprcode: str #hash key
    datestamp: int 
    searchTerms: List[str]  = field(default_factory=list)
    counter: int = 0

    @staticmethod
    def recordResult(searchTerm:str, cprcodes:List[str]):
        
        datestamp = datetime.fromtimestamp(time.time(), tz=pytz.timezone('Asia/Bangkok')).date()
        monthYear = datestamp.strftime("%m-%Y") 
        for cprcode in cprcodes:
            SearchResultMonthlyRecordTable.add(cprcode, monthYear, searchTerm)
            SearchResultDailyRecordTable.add(cprcode, datestamp.strftime("%d-%m-%Y"), searchTerm)

@dataclass
class SearchTermRecord:
    searchTerm: str
    datestamp: int 
    cprcodes: List[str] = field(default_factory=list)
    counter: int = 0

    @staticmethod
    def recordResult(searchTerm: str, cprcodes:List[str]):
        datestamp = datetime.fromtimestamp(time.time(), tz=pytz.timezone('Asia/Bangkok')).date().strftime("%d-%m-%Y")
        SearchTermDailyRecordTable.add(searchTerm, datestamp, cprcodes)


    


In [ ]:
#export
class SearchResultMonthlyRecordTable(Model):
    class Meta:
        table_name = f'search-result-monthly-record-table-{BRANCH}'
        region = 'ap-southeast-1'
        billing_mode='PAY_PER_REQUEST'
        dax_read_endpoints = [DAXENDPOINT] if DAXENDPOINT else None
        dax_write_endpoints = [DAXENDPOINT] if DAXENDPOINT else None
    
    cprcode = UnicodeAttribute(hash_key=True)
    monthYear = UnicodeAttribute(range_key=True) #MMYY
    count = NumberAttribute()
    searchTerms = JSONAttribute(default={})
    
    @classmethod
    def add(cls, cprcode:str, monthYear:int, searchTerm:str):
        try:
            item = next(
                cls.query(cprcode, cls.monthYear==monthYear), 
                cls(cprcode=cprcode, monthYear=monthYear, count = 0, searchTerms=[]))

            item.count += 1
            
            if searchTerm not in item.searchTerms:
                try:
                    item.searchTerms[searchTerm] = 1
                except:
                    item.searchTerms = {searchTerm:1}
            else:
                item.searchTerms[searchTerm] += 1
            item.save()
            return item
        except Exception as e:
            print(f"unable to add {cprcode} to record: {e}")
            # sentry here

class SearchResultDailyRecordTable(Model):
    class Meta:
        table_name = f'search-result-daily-record-table-{BRANCH}'
        region = 'ap-southeast-1'
        billing_mode='PAY_PER_REQUEST'
        dax_read_endpoints = [DAXENDPOINT] if DAXENDPOINT else None
        dax_write_endpoints = [DAXENDPOINT] if DAXENDPOINT else None
    
    cprcode = UnicodeAttribute(hash_key=True)
    datestamp = UnicodeAttribute(range_key=True)
    count = NumberAttribute()
    searchTerms = JSONAttribute()


    @classmethod
    def add(cls, cprcode:str, datestamp:int, searchTerm:str):
        try:
            item = next(
                cls.query(cprcode, cls.datestamp==datestamp), 
                cls(cprcode=cprcode, datestamp=datestamp, count = 0, searchTerms=[]))

            item.count += 1
            if searchTerm not in item.searchTerms:
                try:
                    item.searchTerms[searchTerm] = 1
                except:
                    item.searchTerms = {searchTerm:1}
            else:
                item.searchTerms[searchTerm] += 1
            item.save()
            return item
        except Exception as e:
            print(f"unable to add {cprcode} to record: {e}")
            # sentry here


In [ ]:
#export
class SearchTermDailyRecordTable(Model):
    class Meta:
        table_name = f'search-term-daily-record-table-{BRANCH}'
        region = 'ap-southeast-1'
        billing_mode='PAY_PER_REQUEST'
        dax_read_endpoints = [DAXENDPOINT] if DAXENDPOINT else None
        dax_write_endpoints = [DAXENDPOINT] if DAXENDPOINT else None
    
    searchTerm = UnicodeAttribute(hash_key=True)
    datestamp = UnicodeAttribute(range_key=True)
    count = NumberAttribute()
    cprcodes = ListAttribute()

    @staticmethod
    def appedCprcodes(item, cprcodes:List[str]):
        for code in cprcodes:
            if code not in item.cprcodes:
                item.cprcodes.append(code)

    @classmethod
    def add(cls, searchTerm:str, datestamp:int, cprcodes:List[str]):
        try:
            item = next(
                cls.query(searchTerm, cls.datestamp==datestamp), 
                cls(searchTerm=searchTerm, datestamp=datestamp, count = 0, cprcodes=[]))

            item.count += 1
            for code in cprcodes:
                if code not in item.cprcodes:
                    item.cprcodes.append(code)
            item.save()
            return item
        except Exception as e:
            print(f"unable to add {searchTerm} to record: {e}")
            # sentry here

In [ ]:
#hide
SearchResultMonthlyRecordTable.create_table(billing_mode='PAY_PER_REQUEST')

In [ ]:
#hide
r = SearchResultMonthlyRecordTable.add('1234567890123', 2101, 'cde')
r.searchTerms

{'cde': 3}

In [ ]:
#hide
SearchResultDailyRecordTable.create_table(billing_mode='PAY_PER_REQUEST')

In [ ]:
#hide
r = SearchResultDailyRecordTable.add('1234567890123', 2101, 'd')
r.searchTerms

{'c': 2, 'a': 4, 'd': 1}

In [ ]:
#hide
SearchTermDailyRecordTable.create_table(billing_mode='PAY_PER_REQUEST')

In [ ]:
#hide
r = SearchTermDailyRecordTable.add('abc', 2101, ['234'])
r.count

6

In [ ]:
#hide
SearchResultRecord.recordResult('abc', ['1234567890123', '2345678901234'])

In [ ]:
#hide
SearchTermRecord.recordResult('abc', ['1234567890123', '2345678901234'])

In [ ]:
#hide
SearchResultMonthlyRecordTable.delete_table()
SearchResultDailyRecordTable.delete_table()
SearchTermDailyRecordTable.delete_table()

TableError: Failed to delete table: An error occurred (ResourceNotFoundException) on request (04NO7MMNJK444DKD74UC3CMKD7VV4KQNSO5AEMVJF66Q9ASUAAJG) on table (search-result-monthly-record-table-fakeBranch) when calling the DeleteTable operation: Requested resource not found: Table: search-result-monthly-record-table-fakeBranch not found